# Kaggle BLIP Inference Server + Tunnel
Run a BLIP caption server on Kaggle with a public HTTPS URL for testing the Rust service.

In [ ]:
# Install dependencies (GPU runtime recommended)
!pip -q install fastapi uvicorn transformers pillow accelerate torch cloudflared > /dev/null

In [ ]:
# Optional: Fine-tune BLIP quickly on a small sample (uncomment to run)
# import os, subprocess
# os.makedirs('repo', exist_ok=True)
# %cd /kaggle/working
# %%bash
# python tools/blip_finetune/train_blip.py \#   --output_dir ./blip-finetuned --epochs 1 --batch_size 16 --lr 2e-5 \#   --max_steps 500 --dataset coco_captions --sample 2000


In [ ]:
# Start the BLIP server (uses base model by default).
# To use a fine-tuned model, set MODEL_DIR='./blip-finetuned'
import os, subprocess
os.environ['MODEL_DIR'] = os.getenv('MODEL_DIR', 'Salesforce/blip-image-captioning-base')
proc = subprocess.Popen(['python','-m','uvicorn','tools.blip_infer_server.server:app','--host','0.0.0.0','--port','8001'],
                        stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
print('BLIP server PID', proc.pid)


In [ ]:
# Start Cloudflare quick tunnel and print the public URL
import re, time, subprocess
t = subprocess.Popen(['cloudflared','tunnel','--no-autoupdate','--url','http://127.0.0.1:8001'],
                    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
public_url = None
for _ in range(180):
    line = t.stdout.readline().strip()
    if not line:
        time.sleep(1);
        continue
    m = re.search(r'https://[\w.-]+\.trycloudflare\.com', line)
    if m:
        public_url = m.group(0)
        break
print('PUBLIC_URL =', public_url)
print('Keep this cell running to keep the tunnel alive.')
